# IMPORTS DE LIBRAIRIES

Nous écrivons dans la cellule suivante sur quelle(s) GPU nous souhaitons exécuter le code.  
Cependant, le code utilisé ne supporte pas encore le multi GPU pour l'entraînement.  Pour spécifier plusieurs GPU, séparer leur id d'une virgule.  

In [ ]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0,3"
#os.environ["OMP_NUM_THREADS"] = "1"

In [ ]:
import torch 
print("torch version           : ", torch.__version__)
print("torch cuda version      : ", torch.version.cuda)
print("torch.cuda.is_available : ", torch.cuda.is_available())

In [ ]:
import detectron2
print("detectron2 version : ", detectron2.__version__)

In [ ]:
from detectron2.engine import DefaultPredictor, DefaultTrainer, launch
from detectron2.config import get_cfg, get_stack_cell_config
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data import build_detection_test_loader, build_detection_train_loader
from detectron2.data.common import DatasetFromList
from detectron2.solver import build_lr_scheduler, build_optimizer
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.data.datasets import get_dicts
from detectron2.modeling import build_model
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json, cv2, random, glob

Le logger permet d'afficher des informations importantes tout au long de l'exécution des cellules.  

In [ ]:
from detectron2.utils.logger import setup_logger
setup_logger()

In [ ]:
def imBGRshow(img):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
def imRGBshow(img):
    plt.imshow(img)
    plt.show()

# REGISTER LES IMAGES
## /!\ CHANGE THE DATA PATH ACCORDINGLY

NB : Les classes sont les suivantes :
- 0 : Cellule intacte et nette   (Intact_Sharp)
- 1 : Cellule intacte et floue   (Intact_Blurry)
- 2 : Cellule explosée et nette  (Broken_Sharp)
- 3 : Cellule explosée et floue  (Broken_Blurry)

Pour seulement considérer les cellules nettes, utiliser :
classes = {'Intact_Sharp':0, 'Broken_Sharp':2}

Pour considérer tous les types de cellules, utiliser :
classes = {'Intact_Sharp':0,'Intact_Blurry':1,'Broken_Sharp':2,'Broken_Blurry':3}

In [ ]:
classes = {'Intact_Sharp':0, 'Broken_Sharp':2}
#classes = {'Intact_Sharp':0,'Intact_Blurry':1,'Broken_Sharp':2,'Broken_Blurry':3}

# DATASET

Les noms de fichiers sont enregistrées au format suivant : TxxPyyFzz avec xx le puceron, yy le cluster, et zz le positionnement de l'image des la pile.
Les images sont au format png. Elles sont de taille 400x300. Chaque image possède un fichier json associé ayant le même nom. Dans celui-ci sont enregistrés des données permettant l'identification de l'image, ainsi que les positions et tailles des boîtes englobantes, les classes, ainsi que les différentes segmentations au format polygon.

## AUGMENTATION DES DONNÉES
L'augmentation des données a été réalisée en amont de l'algorithme. Les données augmentées se trouvent d'ores et déjà dans les dossiers spécifiés au dessus. Seules les augmentations en netteté et en cisaillement n'ont pas été implémentées. En effet, la première cause une netteté de toutes les cellules et ne permettra pas au model d'apprendre la différence entre une cellule nette et une cellule floue. Le cisaillement cause quant à lui une déformation trop importante de la membrane extérieure des bactériocytes, rendant très difficile une différenciation des cellules intactes et des cellules explosées.

35 augmentations différentes ont été implémentées. Le réseau verra donc 36 versions des différentes piles. Les données sont séparées en 5 (voir partie organisation ci-dessous).
- 0 : 15444 images => 1404 piles (depuis 39 piles)
- 1 : 15411 images => 1401 piles (depuis 39 piles)
- 2 : 15015 images => 1365 piles (depuis 39 piles)
- 3 : 15015 images => 1365 piles (depuis 39 piles)
- 4 : 14619 images => 1329 piles (depuis 38 piles)

La partie 4 possède 2 défauts majeurs qu'il convient de prendre en compte : la pile Augmented1T9P20 a une image où la moitié est grise. Il convient donc de supprimer cette pile manuellement. Les piles T10P1Fxx ne possèdent pas 11 images. Celles-ci sont écartées automatiquement par l'algorithme lors du chargement des données. Des piles avec un plus grand nombre d'images que celui attendu seraient chargées jusqu'à leur 11ème image seulement.  
Une fois la pile Augmented1T9P20 supprimée manuellement, il reste 14864 images, dont seulement 14619 seront exploitables comme une pile entière doit être écartée, avec toute ses augmentations. C'est cette pile qui amène le nombre de pile à 38 et non 39 comme les autres parties du dataset.  
  
Des défauts moins importants subsitent : nous remarquons que même si les images sont obtenues à partir du même nombre de piles initiallement, il n'y a pas le même nombre d'image dans chaque partie. Certaines augmentations n'ont pas été réalisées sur certaines piles. 

## ORGANISATION
Le dataset est séparé en 3 jeux de données : 
- 60%    => Entraînement
- 20%    => Validation
- 20%    => Test  
  
Les données doivent être rangées dans la structure suivante de fichiers. La variable data_path définie dans la variable suivante doit indiquer l'emplacement du dossier Cross-val.  
/!\ ATTENTION, ce chemin est à adapter.  
└── Cross-val  
&emsp;&emsp;&emsp;   ├── Xval0  
&emsp;&emsp;&emsp; |&emsp;&emsp;   ├── images  
&emsp;&emsp;&emsp; |&emsp;&emsp;   └── labels  
&emsp;&emsp;&emsp;   ├── Xval1  
&emsp;&emsp;&emsp; |&emsp;&emsp;   ├── images  
&emsp;&emsp;&emsp; |&emsp;&emsp;   └── labels  
&emsp;&emsp;&emsp;   ├── Xval2  
&emsp;&emsp;&emsp; |&emsp;&emsp;   ├── images  
&emsp;&emsp;&emsp; |&emsp;&emsp;   └── labels  
&emsp;&emsp;&emsp;   ├── Xval3  
&emsp;&emsp;&emsp; |&emsp;&emsp;   ├── images  
&emsp;&emsp;&emsp; |&emsp;&emsp;   └── labels  
&emsp;&emsp;&emsp;   └── Xval4  
&emsp;&emsp;&emsp; &emsp;&emsp;   ├── images  
&emsp;&emsp;&emsp; &emsp;&emsp;   └── labels  

## VALIDATION CROISEE
Comme son nom l'indique, cette séparation est réalisée afin de pouvoir faire de la validation croisée (cross-validation). Pour des raisons écologiques et de durée d'entraînement, nous n'avons pas tiré profit de cette possibilité, mais il est important de noter qu'elle est facilement implémetable au besoin.  
Un indice indique quelles parties du dataset seront associées avec quel jeu de données (entraînement, validation ou test). Pour réaliser de la validation croisée, il faudra réaliser l'entrainement pour des indices variant de 0 à 4.

In [ ]:
data_path = '/projects/INSA-Image/B01/Data/'
cross_val_idx = 0

In [ ]:
# Modes must have the correct string associated in order to perform the proper operation
mode_train = 'train'
mode_valid = 'val'
mode_test  = 'test'

# By default in our architecture. To use custom names, an override of these names must happen during the configuration (see next section)
dataset_name_train = 'train'
dataset_name_valid = 'val'
dataset_name_test  = 'test'

In [ ]:
# Register the datasets
DatasetCatalog.register(dataset_name_train, lambda: get_dicts(data_path, mode_train, cross_val_idx, classes, dataset_name_train))
DatasetCatalog.register(dataset_name_valid, lambda: get_dicts(data_path, mode_valid, cross_val_idx, classes, dataset_name_valid))
DatasetCatalog.register(dataset_name_test,  lambda: get_dicts(data_path, mode_test,  cross_val_idx, classes, dataset_name_test))

# AFFICHAGE DE QUELQUES IMAGES AVEC LEUR SEGMENTATION MANUELLE

In [ ]:
valid_metadata = MetadataCatalog.get(dataset_name_valid)
valid_dataset_dicts = DatasetCatalog.get(dataset_name_valid)
valid_dataset = DatasetFromList(valid_dataset_dicts, True, 11, serialize=False)
# Attention : si le dataset est serialized alors on ne pourra pas accéder correctement à la liste de tous les dictionnaires de la pile.
# La serialisation (en pickle ici) est cependant très intéressante pour stocker et transmettre des données, ce qui est utile dans d'autres parties du programme

La cellule suivante permet par défaut d'afficher N piles ainsi que leur segmentation. Ces N piles sont tirées au hasard dans le dataset de validation.

In [ ]:
# Visualize N random stacks
N = 1
stack = [None] * 11
for data in random.sample(valid_dataset._lst, N):
    for z in range (11):
        stack[z] = cv2.imread(data[z]["file_name"])
        visualizer = Visualizer(stack[z][:, :, ::-1], metadata=valid_metadata, scale=1)
        out = visualizer.draw_dataset_dict(data[z])
        imRGBshow(out.get_image())
        # print(data[z]["file_name"]) # Print the file path

# ENTRAINEMENT

Detectron2 ne permet pas de réaliser de validation en même temps que l'entraînement. Pour pouvoir éviter le sur entraînement, nous allons réaliser la méthode suivante :
1. Réaliser la configuration
2. Entrainer le modèle en enregistrant ses poids à intervalles réguliers
3. Pour tous les poids enregistrés, évaluer les performances du réseau afin de trouver les meilleurs poids
4. Enfin, pour les poids sélectionnés, nous allons réaliser l'évaluation de notre réseau entraîné.


### ENTRAINEMENT MULTI-GPU
L'entraînement multi-GPU produit une erreur dans ce notebook. Cependant, il est bien possible d'entraîner le réseau en multi-GPU. Pour cela, il faut utiliser la commande suivante dans le terminal, en étant dans le dossier dans lequel le git est cloné, avec un environnement adéquat :  
OMP_NUM_THREADS=1 python tools/train_net.py \
    --config-file configs/Segmentation-Z/mask_rcnn_z_50.yaml \
    --num-gpus 2 \
    --dist-url "auto" \
    OUTPUT_DIR training_dir  
Le bug doit donc venir de la manière dont Jupyter interagit avec le processus de spawning utilisé pour réunir les informations venant des différentes GPU. La suite de ce notebook explique donc les idées derrière le code du fichier train_net.py. Il faudra laisser la variable num_gpus dans ce notebook à 1.

## CONFIGURATION DU RESEAU ET DES PARAMETRES D'ENTRAINEMENT
Cette configuration est similaire que pour des utilisations de detectron2 normales.  
Il faut cependant changer les configurations par défaut suivantes :
- Architecture
- Input chargé par le dataloader
- Nombre de classes
- Poids du réseaux et couches figées
- Solveur avec enregistrement des poids
- Nombre de GPU (seulement 1 GPU est possible pour l'instant)
- Automatic Mixed Precision (debug en cours)
  
NB : Selon la configuration donnée par la fonction get_stack_cell_config, les images seront redimensionnées de 300x400 à 480x640. Cela permet d'être à la même taille que les images du challenge COCO. C'est aussi une taille divisible par le downsampling (sous-echantillonnage) du resnet.

In [ ]:
#config_architecture_file = '../configs/Segmentation-Z/mask_rcnn_z_50.yaml'
config_architecture_file = '../configs/Segmentation-Z/mask_rcnn_3d.yaml'

# Pour sauvegarder des données d'entrapinement, notamment les poids du réseau entraîné
output_directory = "/tmp/TEST/outputs/0/"

In [ ]:
num_gpus = 1      # torch.cuda.device_count()

In [ ]:
# Configuration de base présente dans ../detectron2/config/defaults.py
cfg = get_cfg()

# Configuration de l'architecture (depuis le fichier de configuration défini dans config_architecture_file)
cfg.merge_from_file(config_architecture_file)
cfg.MODEL.RESNETS.DEPTH = 18                                 # Configuration of the depth of the resnet network, default to 50, for lighter models, use 18
cfg.MODEL.RESNETS.RES2_OUT_CHANNELS = {18:64, 32:64, 50:256, 101:256, 152:256}[cfg.MODEL.RESNETS.DEPTH]

# Configuration de l'input : pile. Pour d'autres configuration de pile, il faut soit override les paramètres particuliers configurés dans la fonction, soit écrire une autre fonction.
cfg = get_stack_cell_config(cfg)

# Configuration du nombre de classes
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)

# Configuration des poids du réseau et des couches figées
cfg.MODEL.BACKBONE.FREEZE_AT = 0      # 0  => aucune couche figée
cfg.MODEL.WEIGHTS = ""                # "" => pas de poids préchargés, ils seront tirés au hasard

# Configuration du solveur
cfg.SOLVER.IMS_PER_BATCH = 1          # Attention à la taille de la mémoire dont dispose la GPU, doit aussi être un multiple du nombre de GPU
if (cfg.SOLVER.IMS_PER_BATCH % num_gpus != 0):    # Pour être sûr d'être divisible par le nombre de GPU
    cfg.SOLVER.IMS_PER_BATCH = (cfg.SOLVER.IMS_PER_BATCH // num_gpus) * num_gpus
cfg.SOLVER.MAX_ITER = 100      # Pour test, sinon, remettre 10000
cfg.SOLVER.CHECKPOINT_PERIOD = cfg.SOLVER.MAX_ITER // 20
cfg.SOLVER.BASE_LR = 0.001

# Configuration du dossier pour sauvegarder les sorties de l'algorithme
cfg.OUTPUT_DIR = output_directory

# Configuration multi GPU
cfg.SOLVER.REFERENCE_WORLD_SIZE = num_gpus

# Configuration automatic mixed precision (False => float32, True => float16)
cfg.MODEL.USE_AMP = False

## Configuration de l'évaluation en même temps que les poids sont enrégistrés
#cfg.TEST.EVAL_PERIOD = cfg.SOLVER.CHECKPOINT_PERIOD
# MAIS LE FAIT SUR LE DATASET TEST AU LIEU DE VALID...

# La configuration ne pourra plus être modifiée :
cfg.freeze()

## ENTRAINEMENT DU RESEAU

In [ ]:
def my_train():
    trainer = DefaultTrainer(cfg)
    trainer.resume_or_load(resume=False)
    # False to begin training from scratch, 
    # True, takes the specified weights in config, or begin from scratch if no weight specified
    # In our case, since we didn't specify weights, trianing will begin from scratch
    return trainer.train()

if __name__ == "__main__":
    launch(my_train, num_gpus_per_machine=num_gpus, num_machines=1, dist_url="auto")

### /!\ 
Il faut bien ré écrire le chemin présent dans output_directory entre les guillemets afin d'afficher les courbes. Il est normal que les losses soient 0 pour les images proches des extrémités car elles ne possèdent en général aucune cellule nette et en ne prédisant rien, elles ne se trompent jamais ou presque. 
  
Ceci est une limitation du réseau, que nous attendions dans le cas de convolutions 2D. Pour le backbone réalisant des convolutions 3D, toutes les images sont traitées de manière plus similaire et cette limitation est moins importantes.  
  
Pour les backbones réalisant des convolutions 2D, nous pourrions imaginer mélanger les images au sein d'une même pile afin que l'apprentissage se réalise aussi sur les images des bords. Cependant, quand l'information sera mélangée au sein du réseau, nous perdons l'organisation spatiale réelle des images, ce qui perturbera l'attention entre features de chaque image.

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir "/tmp/TEST/outputs/3D"

## VALIDATION DES MEILLEURS POIDS DU RESEAU
Pour tous les poids enregistrés, nous évaluons les performances du réseau avec le dataset de validation afin de trouver les meilleurs poids.

In [ ]:
cfg.defrost()

In [ ]:
all_weights_paths = sorted(glob.glob(os.path.join(output_directory, 'model_*.pth')))
metrics = [None] * len(all_weights_paths)
for (i, weights_path) in zip(range(len(all_weights_paths)), all_weights_paths):
    print(weights_path)
    cfg.MODEL.WEIGHTS = weights_path
    predictor = DefaultPredictor(cfg)
    valid_evaluator = COCOEvaluator(dataset_name_valid, cfg, True, output_dir=output_directory)
    valid_loader = build_detection_test_loader(cfg, dataset_name_valid)
    # Faire un truc qui garde en mémoire tout et prenne le meilleur
    metrics[i] = inference_on_dataset(predictor.model, valid_loader, valid_evaluator, cfg.DATALOADER.IS_STACK)
    #print(weights_path)

In [ ]:
output_directory = "/tmp/TEST/outputs/3D/"
#output_directory = output_directory
best_weights = "model_final.pth"

## EVALUATION DU MODELE
Pour les poids trouvés précédemment, nous évaluons notre modèle sur le dataset de test.  
Cette évaluation ne peut se faire avec le dataset précédent car c'est celui qui a servi à la sélection des poids.

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(output_directory, best_weights)
# OR cfg.MODEL.WEIGHTS = best_weights
# IF best_weights has the whole paths
# Training on Detectron2 with a Validation set, and plot loss on it to avoid overfitting

In [ ]:
test_evaluator = COCOEvaluator(dataset_name_test, cfg, True, output_dir=output_directory)
test_loader = build_detection_test_loader(cfg, dataset_name_test)
test_metrics = inference_on_dataset(predictor.model, test_loader, test_evaluator, cfg.DATALOADER.IS_STACK)

# VISUALISATION SUR UNE STACK

In [ ]:
test_metadata = MetadataCatalog.get(dataset_name_test)
test_dataset_dicts = DatasetCatalog.get(dataset_name_test)
test_dataset = DatasetFromList(test_dataset_dicts, cfg.DATALOADER.IS_STACK, cfg.INPUT.STACK_SIZE, serialize=False)
# Attention : si le dataset est serialized alors on ne pourra pas accéder correctement à la liste de tous les dictionnaires de la pile.
# La serialisation (en pickle ici) est cependant très intéressante pour stocker et transmettre des données, ce qui est utile dans d'autres parties du programme

In [ ]:
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)

In [ ]:
N = 1
stack = [None] * cfg.INPUT.STACK_SIZE
out   = [None] * cfg.INPUT.STACK_SIZE
for data in random.sample(test_dataset._lst, N):
    for z in range(cfg.INPUT.STACK_SIZE):
        stack[z] = cv2.imread(data[z]["file_name"])
    outputs = predictor(stack)
    
    for z in range(cfg.INPUT.STACK_SIZE):
        visualizer = Visualizer(stack[z][:, :, ::-1], metadata=test_metadata, scale=1)
        visualizer_GT = Visualizer(stack[z][:, :, ::-1], metadata=test_metadata, scale=1)
        out[z] = visualizer.draw_instance_predictions(outputs[z]["instances"].to("cpu"))
        out_GT = visualizer_GT.draw_dataset_dict(data[z])
        print(z)
        print("Ground Truth")
        imRGBshow(out_GT.get_image())
        print("Predicted")
        imRGBshow(out[z].get_image())